### Please upload the json file and specify the path

In [52]:
import json

with open("/kaggle/input/test-input/input.json", "r") as file:
    input_data = json.load(file)

# Installing Dependencies

In [ ]:
!pip install python-dotenv numpy pandas scikit-learn sentence-transformers huggingface-hub hf-xet langchain-groq matplotlib seaborn nltk streamlit transformers -q
!pip install langchain langchain-community ipykernel google-search-results json_repair -q


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
# import streamlit as st
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, GPT2LMHeadModel, GPT2Tokenizer
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from typing import Dict, List
from sentence_transformers import SentenceTransformer, util
import json
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
import requests
import math
from langchain_community.utilities import SerpAPIWrapper
from json_repair import repair_json
import json_repair

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/torch_xla/__init__.py:251: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
E0000 00:00:1757849219.394438      10 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: === 
learning/45eac/tfrc/runtime/common_lib.cc:230


In [3]:
nltk.download("punkt")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# Main workflow

In [35]:
prompts = [input_data['prompt']]
agent_responses = [d for d in input_data['agent_responses']]

prompts = prompts * len(agent_responses)

In [36]:
def my_func(prompt, agent_response, groq_api_key, serpapi_api_key):

    def load_semantic_model():
        return SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

    def load_nli_model():
        tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-mnli")
        model_nli = AutoModelForSequenceClassification.from_pretrained("facebook/bart-large-mnli")
        return tokenizer, model_nli

    def load_gpt2_for_fluency():
        model_for_fluency = GPT2LMHeadModel.from_pretrained("gpt2")
        tokenizer_for_fluency = GPT2Tokenizer.from_pretrained("gpt2")
        model_for_fluency.eval()
        return model_for_fluency, tokenizer_for_fluency

    def check_factuality(prompt, agent_response, groq_api_key, serpapi_key, max_claims=5, max_results=2):
        model = ChatGroq(model='gemma2-9b-it', api_key=groq_api_key)
        model2 = ChatGroq(model='llama-3.3-70b-versatile', api_key=groq_api_key)
        search = SerpAPIWrapper(serpapi_api_key=serpapi_key, params={"num": str(max_results)})

        template1 = ChatPromptTemplate([
            ("system", "You are a brilliant assistant."),
            ("human", """[KB-Based QA]
    You are given a piece of text that includes knowledge claims. A claim is a statement that asserts something as true or false, which can be verified by humans.

    [Task]
    Your task is to accurately identify and extract up to {k} claims stated in the provided text. 
    Then, resolve any coreference (pronouns or other referring expressions) in the claim for clarity. Each claim should be concise (less than 15 words) and self-contained.
    Your response MUST be a list of dictionaries. Each dictionary should contain the key "claim", which corresponds to the extracted claim (with all coreferences resolved). 
    You MUST only respond in the format as described below. DO NOT RESPOND WITH ANYTHING ELSE. ADDING ANY OTHER EXTRA NOTES THAT VIOLATE THE RESPONSE FORMAT IS BANNED. START YOUR RESPONSE WITH '['.
    Include **at most {k} claims**. If the text contains fewer than {k} claims, output only the available claims.

    [Response Format]
    [{{"claim": "Ensure that the claim is fewer than 15 words and conveys a complete idea. Resolve any coreference (pronouns or other referring expressions) in the claim for clarity."}}, ... ]

    Here are two examples:
    [text]:
    Tomas Berdych defeated Gael Monfis 6-1, 6-4 on Saturday. The sixth-seed reaches Monte Carlo Masters final for the first time. Berdych will face either Rafael Nadal or Novak Djokovic in the final.
    [response]:
    [{{"claim": "Tomas Berdych defeated Gael Monfis 6-1, 6-4"}}, {{"claim": "Tomas Berdych reaches Monte Carlo Masters final"}}, {{"claim": "Tomas Berdych is the sixth-seed"}}, {{"claim": "Tomas Berdych will face either Rafael Nadal or Novak Djokovic"}}, {{"claim": "Berdych will face either Rafael Nadal or Novak Djokovic in the final"}}]

    [text]:
    Tinder only displays the last 34 photos - but users can easily see more. Firm also said it had improved its mutual friends feature.
    [response]:
    [{{"claim": "Tinder only displays the last 34 photos"}}, {{"claim": "Tinder users can easily see more photos"}}, {{"claim": "Tinder said it had improved its mutual friends feature"}}]

    Now complete the following:
    [text]:
    {input_text}
    [response]:
    """)
        ])

        template2 = ChatPromptTemplate([
            ('system', "You are a brilliant assistant."),
            ('human', """[KB-based QA]
    You are a query generator designed to help users verify a given claim using search engines. 
    Your primary task is to generate a **valid JSON array** containing **one effective and skeptical search engine query**. 
    This query should assist users in critically evaluating the factuality of a provided claim using search engines. 

    [Important Instructions]
    1. You MUST respond **only** in strict JSON format. 
    2. The JSON array must use **double quotes** for strings, e.g., ["query here"].
    3. Single quotes (' ') or Python-style lists are invalid and not allowed.
    4. Do not include any extra text, comments, or explanations. 
    5. The array must contain exactly **one query**.
    6. Start the response directly with `[` and end with `]`.

    [response format]:
    ["query"]

    Here are 3 examples: 
    [claim]: The CEO of Twitter is Bill Gates. 
    [response]: ["Who is the CEO of Twitter?"]

    [claim]: Michael Phelps is the most decorated Olympian of all time. 
    [response]: ["Who is the most decorated Olympian of all time?"]

    [claim]: ChatGPT is created by Google. 
    [response]: ["Who created ChatGPT?"]

    Now complete the following: 
    [claim]: {input} 
    [response]:
    """)
        ])
        
        template3 = ChatPromptTemplate([
            ('system', "You are a brilliant assistant."),
            ('human', """[KB-based QA]
You are given a piece of text. Your task is to identify whether there are any factual errors within the text.
When you are judging the factuality of the given text, you could reference the provided evidences if needed. 
The provided evidences may be helpful. Some evidences may contradict to each other. 
You must be careful when using the evidences to judge the factuality of the given text.
The response should be a dictionary with six keys - "reasoning", "factuality", "error", "out of context", "correction", and "factually incorrect number of sentence", which correspond to the reasoning, whether the given text is factual or not (Boolean - True or False), the factual error present in the text, any assumptions or parts that are out of context, the corrected text, and the number of the sentence that are factually incorrect.

The following is the given text : {answer} 
The following is the provided evidences : {evidences} 
You should only respond in the format as described below. DO NOT RETURN ANYTHING ELSE.
START YOUR RESPONSE WITH '{{'. 

[response format]: 
{{"reasoning": "Why is the given text factual or non-factual? Be careful when you said something is non-factual. When you said something is non-factual, you must provide multiple evidences to support your decision.", 
"error": "None if the text is factual; otherwise, describe the error.",
"correction": "The corrected text if there is an error.",
"out-of-context": "Parts of sentences that were out of context.",
"factually incorrect number of sentences": "A whole number indicating the number of the sentences that are factually incorrect.",
"factuality": "True if the given text is factual, False otherwise."}}
""")
        ])

        prompt1 = template1.invoke({"input_text": agent_response, "k": str(max_claims)})
        result = model.invoke(prompt1)
        try:
            claims = json_repair.loads(result.content)
        except json.JSONDecodeError as e:
            return 0.0, {"error": "Failed to parse claims"}, claims, []
            
            # st.markdown("##### Extracted Claims:")
            # st.json(claims)
        
        evidences = []
        for claim in claims:
            try:
                prompt2 = template2.invoke({'input': claim['claim']})
                response = model.invoke(prompt2)
                query = json.loads(response.content.replace("\\'", "'"))
                
                res = search.results(query[0])
                if "knowledge_graph" in res:
                    if "description" in res['knowledge_graph']:
                        evidences.append(res['knowledge_graph']['description'])
                
                for result_snippet in res.get('organic_results', [])[:max_results]:
                    evidences.append(result_snippet['snippet'])
            except Exception as e:
                print(e)
                continue
            
        try:
            prompt3 = template3.invoke({"answer": agent_response, "evidences": evidences})
            result = model2.invoke(prompt3)
            
            # Clean the response content
            content = result.content.strip()

            final_evaluation = json_repair.loads(content)
            
        except Exception as e:
            return 0.0, {"error": f"Failed to evaluate factuality: {str(e)}"}, claims, evidences

        try:
            total_sentences = len([s for s in agent_response.split('.') if s.strip()])
            if total_sentences == 0:
                total_sentences = 1
            
            # incorrect_key = 'factually_incorrect_number_of_sentences'
            # if incorrect_key not in final_evaluation:
            #     for alt_key in ['factually incorrect number of sentences', 'incorrect_sentences', 'factually_incorrect_sentences']:
            #         if alt_key in final_evaluation:
            #             incorrect_key = alt_key
            #             break
            #     else:
            #         final_evaluation[incorrect_key] = 0
            
            incorrect_sentences = int(final_evaluation["factually incorrect number of sentences"])
            fact_error = incorrect_sentences / total_sentences
            factual_accuracy_score = 1 - fact_error
            
        except Exception as e:
            factual_accuracy_score = 0.0
        
        return factual_accuracy_score, final_evaluation, claims, evidences

    model = load_semantic_model()
        
    def semantic_score(prompt_hint, response):
        emb_pr = model.encode(prompt_hint, convert_to_tensor=True)
        emb_res = model.encode(response, convert_to_tensor=True)
        sim_score = util.cos_sim(emb_pr, emb_res).item()
        return (sim_score + 1) / 2.0

    semantic_s = semantic_score(prompt, agent_response)


    tokenizer, model_nli = load_nli_model()
        
    def nli_entailment_prob(premise, hypothesis):
        inputs = tokenizer(premise, hypothesis, return_tensors="pt", truncation=True)
        logits = model_nli(**inputs).logits
        probs = F.softmax(logits, dim=-1)[0].detach().cpu().numpy()
        entail_prob = probs[2]
        neutral_prob = probs[1]
        contradict_prob = probs[0]
        return float(entail_prob), float(neutral_prob), float(contradict_prob)

    ent, neu, cont = nli_entailment_prob(prompt, agent_response)

    # col_nli_1, col_nli_2, col_nli_3 = st.columns(3)
    # col_nli_1.metric("Entailment", f"{ent:.2f}")
    # col_nli_2.metric("Neutrality", f"{neu:.2f}")
    # col_nli_3.metric("Contradiction", f"{cont:.2f}")


    def nli_entailment_prob_bw_sentences(premise, hypothesis):
        if not premise.strip() or not hypothesis.strip():
            return 0.0, 0.0, 0.0
        inputs = tokenizer(premise, hypothesis, return_tensors="pt", truncation=True, padding=True, max_length=512)
        logits = model_nli(**inputs).logits
        probs = F.softmax(logits, dim=-1)[0].detach().cpu().numpy()
        contradict_prob = probs[0]
        neutral_prob = probs[1]
        entail_prob = probs[2]
        return float(contradict_prob), float(neutral_prob), float(entail_prob) 

    def sequential_entailment(response):
        sentences = nltk.sent_tokenize(response)
        sentences = [s.strip() for s in sentences if s.strip()]
        
        entail_scores = []
        neutral_scores = []
        contradict_scores = []
        
        for i in range(len(sentences) - 1):
            c, n, e = nli_entailment_prob_bw_sentences(sentences[i], sentences[i+1])
            entail_scores.append(e)
            neutral_scores.append(n)
            contradict_scores.append(c)
        
        return sentences, entail_scores, neutral_scores, contradict_scores

    sentences, entail, neutral, contradict = sequential_entailment(agent_response)

    coherent_scores = [0,0,0]
    if len(entail) > 0:
        overall_neutral_score = sum(neutral) / len(neutral)
        overall_entail_score = sum(entail) / len(entail)
        overall_contradict_score = sum(contradict) / len(contradict)
        coherent_scores = [overall_neutral_score, overall_entail_score, overall_contradict_score]

    def lexical_diversity(text):
        tokens = nltk.word_tokenize(text.lower())
        total = len(tokens)
        unique = len(set(tokens))
        return unique / total if total > 0 else 0

    def distinct_n(text, n=2):
        tokens = nltk.word_tokenize(text.lower())
        ngrams = [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]
        return len(set(ngrams)) / len(ngrams) if ngrams else 0

    unigrams_diversity = lexical_diversity(agent_response)
    bigrams_diversity = distinct_n(agent_response)


    model_for_fluency, tokenizer_for_fluency = load_gpt2_for_fluency()
        
    def calculate_perplexity(text):
        encodings = tokenizer_for_fluency(text, return_tensors="pt")
        with torch.no_grad():
            outputs = model_for_fluency(**encodings, labels=encodings["input_ids"])
            loss = outputs.loss
        return math.exp(loss.item())

    def normalized_fluency(text):
        try:
            ppl = calculate_perplexity(text)
            score = 1 / (1 + math.log(1 + ppl))
            return score, ppl
        except Exception as e:
            # st.error(f"Error calculating perplexity: {e}")
            return 0.0, float('inf')

    score, ppl = normalized_fluency(agent_response)
            
    def grammar_check_languagetool(text):
        url = "https://api.languagetool.org/v2/check"
        data = {
            "text": text,
            "language": "en-US"
        }
        try:
            response = requests.post(url, data=data).json()
            errors = response.get("matches", [])
            score = 1 - (len(errors) / max(1, len(text.split())))
            return max(0, score), errors
        except requests.exceptions.RequestException as e:
            return 0.0, []
        except Exception as e:
            return 0.0, []

    grammar_score, grammar_errors = grammar_check_languagetool(agent_response)

    def parse_constraints(prompt_text: str) -> Dict[str, List[str]]:
        constraints = {
            "quantity": [], "format": [], "length": [], "style": [], "do_dont": []
        }
        text = prompt_text.lower().strip()

        quantity_patterns = [r"(?:list|give|provide|generate|mention)\s+(\d+)\s",
                            r"(?:list|give|provide|generate|mention)\s+(one|two|three|four|five|six|seven|eight|nine|ten)\s"]
        word_to_num = {"one": 1, "two": 2, "three": 3, "four": 4, "five": 5, "six": 6, "seven": 7, "eight": 8, "nine": 9, "ten": 10}
        for pattern in quantity_patterns:
            match = re.search(pattern, text)
            if match:
                value = match.group(1)
                constraints["quantity"].append(int(value) if value.isdigit() else word_to_num.get(value))

        format_keywords = ["json", "table", "bullet points", "bullets", "list", "yaml", "csv", 'markdown', 'dict', 'dictionary', 'paragraph', 'monolithic']
        for keyword in format_keywords:
            if keyword in text:
                constraints["format"].append(keyword)

        length_patterns = [r"limit to (\d+) words", r"not more than (\d+) words", r"no more than (\d+) words",
                            r"less than (\d+) words", r"greater than (\d+) words", r"not more than (\d+) sentences",
                            r"up to (\d+) words"]
        for pattern in length_patterns:
            match = re.search(pattern, text)
            if match:
                constraints["length"].append(match.group(0))

        style_keywords = ["formal tone", "informal tone", "past tense", "present tense", "active voice", "passive voice",
                            'easy language', 'not ai generated', 'human generated', 'technical jargons']
        for keyword in style_keywords:
            if keyword in text:
                constraints["style"].append(keyword)

        dont_pattern = r"(?:do not|don't)\s+(?:mention|include|talk about)\s+([\w\s]+)"
        dont_match = re.findall(dont_pattern, text)
        if dont_match:
            constraints["do_dont"].extend([item.strip() for item in dont_match])

        return constraints

    parsed_constraints = parse_constraints(prompt)

    if groq_api_key:
        try:
            model = ChatGroq(model='openai/gpt-oss-20b', api_key=groq_api_key, temperature=1)

            judge_prompt = ChatPromptTemplate.from_messages([
                ("system",
                    """You are an **evaluation model**. Your task is to **strictly evaluate** if the AGENT RESPONSE follows the USER PROMPT and the provided CONSTRAINTS.
                    ### USER PROMPT: {user_prompt}
                    ### CONSTRAINTS (rules that must be followed exactly): {constraints}
                    ### AGENT RESPONSE: {agent_response}
                    ### EVALUATION RULES:
                    1. Assign a score between 0 and 1 for each constraint type(`quantity`, `format`, `length`, `style`, `do_dont`):
                    2. **final_score** = Average of all five scores.
                    3. Your entire output **MUST be valid JSON only**. Do **NOT** include any extra text or explanation outside of the JSON.
                    ### JSON OUTPUT FORMAT:
                    {{
                    "quantity_score": float,
                    "format_score": float,
                    "length_score": float,
                    "style_score": float,
                    "do_dont_score": float,
                    "final_score": float,
                    "reasoning": "One short paragraph explaining why these scores were given."
                    }}
                    """
                )
            ])

            final_prompt = judge_prompt.format(
                user_prompt=prompt,
                constraints=json.dumps(parsed_constraints, indent=2),
                agent_response=agent_response
            )

            result = model.invoke(final_prompt)

            try:
                eval_data = json.loads(result.content)
                # col_s1, col_s2, col_s3 = st.columns(3)
                # col_s4, col_s5, col_final = st.columns(3)

                # col_s1.metric("Quantity Score", f"{eval_data.get('quantity_score', 0):.2f}")
                # col_s2.metric("Format Score", f"{eval_data.get('format_score', 0):.2f}")
                # col_s3.metric("Length Score", f"{eval_data.get('length_score', 0):.2f}")
                # col_s4.metric("Style Score", f"{eval_data.get('style_score', 0):.2f}")
                # col_s5.metric("Do/Don't Score", f"{eval_data.get('do_dont_score', 0):.2f}")
                # col_final.metric("Final Score", f"{eval_data.get('final_score', 0):.2f}")
                
                # st.markdown("**Reasoning:**")
                # st.write(eval_data.get('reasoning', 'No reasoning provided.'))

            except json.JSONDecodeError as e:
                print(e)
        except Exception as e:
            # st.error(f"An error occurred with the LLM call. Please check your API key and network connection: {e}")
            print(e)
    else:
        # st.warning("Please provide a Groq API key to run the instruction adherence check.")
        print("error")


    if groq_api_key and serpapi_api_key:
        try:
            factual_accuracy_score, final_evaluation, claims, evidences = check_factuality(
                prompt, agent_response, groq_api_key, serpapi_api_key
            )        
        except Exception as e:
            print(e)
    else:
        print("Warning")
    result = {
        "prompt": prompt,
        "agent_response": agent_response,
        "AI_as_judge_results": final_evaluation,
        "factual_accuracy_score": factual_accuracy_score,
        "claims": claims,
        "evidences": evidences,
        "parsed_constraints": parsed_constraints,
        "instruction_adherence": eval_data,
        "semantic_score": semantic_s,
        "entail_prob": ent,
        "neutral_prob": neu,
        "contradict_prob": cont,
        "coherent_scores_overall_enc": coherent_scores,
        "unigrams_diversity": unigrams_diversity,
        "bigrams_diversity": bigrams_diversity,
        "normalized_perplexity": score,
        "raw_perplexity": ppl,
        "grammar_score": grammar_score,
        "grammar_errors": grammar_errors,
        "entail_scores": entail,
        "neutral_scores": neutral,
        "contradict_scores": contradict
    }

    result = json.loads(repair_json(json.dumps(result)))

    return result

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
groq_api_key = os.getenv['GROQ_API_KEY']
serpapi_api_key = os.getenv['SERP_API_KEY']

In [38]:
import os
import jax

print("TPU devices:", jax.devices("tpu"))
print("CPU cores available:", os.cpu_count())

TPU devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1), TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1), TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1), TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]
CPU cores available: 96


In [39]:
prompts

['What are the assumptions of Linear Regression',
 'What are the assumptions of Linear Regression',
 'What are the assumptions of Linear Regression',
 'What are the assumptions of Linear Regression',
 'What are the assumptions of Linear Regression',
 'What are the assumptions of Linear Regression',
 'What are the assumptions of Linear Regression',
 'What are the assumptions of Linear Regression']

# Running the model

In [40]:
from concurrent.futures import ProcessPoolExecutor
import os

def process_item(x, prompt, agent_response, groq_api_key, serpapi_api_key):
    print(f"Processing {x} in PID {os.getpid()}", end="")
    return my_func(prompt, agent_response, groq_api_key, serpapi_api_key)

if __name__ == "__main__":
    indices = list(range(len(prompts)))

    with ProcessPoolExecutor() as executor:
        results = list(executor.map(
            process_item,
            indices,
            prompts,
            agent_responses,
            [groq_api_key]*len(prompts),
            [serpapi_api_key]*len(prompts)
        ))

        results = dict(zip(indices, results))

Processing 0 in PID 4032Processing 2 in PID 4034Processing 1 in PID 4033Processing 5 in PID 4037Processing 6 in PID 4038Processing 3 in PID 4035Processing 4 in PID 4036Processing 7 in PID 4039

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
Passing a tuple of `past_key_values` is deprecated a

In [41]:
results

{0: {'prompt': 'What are the assumptions of Linear Regression',
  'agent_response': 'Linear regression is based on several key assumptions that ensure the model is valid and reliable. First, there is the assumption of linearity, which means the relationship between the independent and dependent variables is linear. Second, the independence of observations assumes that each data point is independent of the others. Third, homoscedasticity requires that the variance of the error terms is constant across all levels of the independent variable. Fourth, the normality of errors assumes that the residuals are normally distributed. Fifth, there should be no multicollinearity between the independent variables, meaning they should not be highly correlated with each other. Additionally, linear regression assumes that the model is correctly specified, meaning all relevant variables are included and no important variables are omitted. However, it is also assumed that the model can handle non-linear 

In [42]:
!pip install streamlit -q


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [43]:
!mkdir pages

mkdir: cannot create directory ‘pages’: File exists


In [44]:
%%writefile app.py
import streamlit as st
import os
os.system("pip install matplotlib")
import matplotlib.pyplot as plt
import json

with open("data.json", "r") as f:
    data = json.load(f)
st.set_page_config(layout="wide")
st.title("LLM Response Evaluation Dashboard")
st.write("This dashboard evaluates and scores responses of LLM models, and checks it for semantic similarity, NLI-based entailment, adherence to specific instructions, coherency in responses and factual accuracy.✅")

st.header("Overall Trends")
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')

fig, axs = plt.subplots(2, 2, figsize=(12,8))
# fig.suptitle("Overall Trends", fontsize=16)
n = list(range(1, len(data)+1))

axs[0][0].plot(n, [d["semantic_score"] for d in data.values()])
axs[0][0].set_title("Semantic Scores")
axs[0][0].set_xticks(n)
avg_semantic = sum([d["semantic_score"] for d in data.values()]) / len(data)
axs[0][0].axhline(y=avg_semantic, linestyle='--', label=f'Average Semantic Score: {avg_semantic:.3f}')
axs[0][0].legend()

axs[0][1].plot(n, [d["factual_accuracy_score"] for d in data.values()])
axs[0][1].set_title("Factual Accuracy")
axs[0][1].set_xticks(n)
avg_factuality = sum([1 if d["factual_accuracy_score"] else 0 for d in data.values()]) / len(data)
axs[0][1].axhline(y=avg_factuality, linestyle='--', label=f'Average Factual Accuracy: {avg_factuality:.3f}')
axs[0][1].legend()

axs[1][0].plot(n, [d["normalized_perplexity"] for d in data.values()])
axs[1][0].set_title("Normalized Perplexity Score")
axs[1][0].set_xticks(n)
avg_normalized_ppx = sum([d["normalized_perplexity"] for d in data.values()]) / len(data)
axs[1][0].axhline(y=avg_normalized_ppx, linestyle='--', label=f'Average Normalized Perplexity: {avg_normalized_ppx:.3f}')
axs[1][0].legend()

axs[1][1].plot(n, [d["grammar_score"] for d in data.values()])
axs[1][1].set_title("Grammar Score")
axs[1][1].set_xticks(n)
avg_grammar_score = sum([d["grammar_score"] for d in data.values()]) / len(data)
axs[1][1].axhline(y=avg_grammar_score, linestyle='--', label=f'Average Grammar Score: {avg_grammar_score:.3f}')
axs[1][1].legend()

plt.tight_layout()
st.pyplot(fig)

# st.metric("Factuality Rate", f"{avg_factuality*100:.2f}%")

st.header("Individual Responses")

for key in data.keys():
    st.markdown(f"[View Response {key}](Response?key={key})")

Overwriting app.py


In [ ]:
%%writefile pages/2_Response.py
import streamlit as st
import os
os.system("pip install matplotlib numpy")
import matplotlib.pyplot as plt
import numpy as np
import json

# key = st.session_state.get("key", "1")
key = st.query_params.key
key = key

with open("data.json", "r") as f:
    data = json.load(f)[key]

st.header("Analysis Results")
st.subheader("1.1. Semantic Search Score")

st.metric("Semantic Score (0-1, closer to 1 is better)", f"{data['semantic_score']:.2f}")
st.info("Measures how semantically similar the response is to the prompt.")

st.subheader("1.2. Prompt-Response Entailment")
col_nli_1, col_nli_2, col_nli_3 = st.columns(3)
col_nli_1.metric("Entailment", f"""{data["entail_prob"]:.2f}""")
col_nli_2.metric("Neutrality", f"""{data["neutral_prob"]:.2f}""")
col_nli_3.metric("Contradiction", f"""{data["contradict_prob"]:.2f}""")
st.info("Measures the logical relationship between the prompt (premise) and the response (hypothesis).")

st.subheader("1.3. Sequential Sentence Coherence")
st.info("Measures the logical coherence between adjacent sentences. High entailment indicates a smooth, consistent flow.")
col_seq_1, col_seq_2, col_seq_3 = st.columns(3)
col_seq_1.metric("Overall Entailment", f"""{data["coherent_scores_overall_enc"][0]:.2f}""")
col_seq_2.metric("Overall Neutrality", f"""{data["coherent_scores_overall_enc"][1]:.2f}""")
col_seq_3.metric("Overall Contradiction", f"""{data["coherent_scores_overall_enc"][2]:.2f}""")

# Plotting the results
sentences = data["agent_response"].split('.')

entail = data['entail_scores']
neutral = data['neutral_scores']
contradict = data['contradict_scores']

pairs = [f"S{i+1} -> S{i+2}" for i in range(len(entail))]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Stacked bar chart

ax1.bar(pairs, contradict, label="Contradiction", color="red")
ax1.bar(pairs, neutral, bottom=contradict, label="Neutral", color="gray")
ax1.bar(pairs, entail, bottom=[c + n for c, n in zip(contradict, neutral)], label="Entailment", color="green")
ax1.set_ylim(0, 1)
ax1.set_ylabel("Probability")
ax1.set_xlabel("Sentence Pair")
ax1.set_title("Entailment vs Neutral vs Contradiction (Sequential)")
ax1.legend()
ax1.tick_params(axis='x', rotation=45)

# Grouped bar chart
x = np.arange(len(pairs))
width = 0.25
ax2.bar(x - width, contradict, width, label="Contradiction", color="red")
ax2.bar(x, neutral, width, label="Neutral", color="gray")
ax2.bar(x + width, entail, width, label="Entailment", color="green")
ax2.set_xticks(x)
ax2.set_xticklabels(pairs, rotation=45)
ax2.set_ylim(0, 1)
ax2.set_ylabel("Probability")
ax2.set_xlabel("Sentence Pair")
ax2.set_title("Sequential Sentence Relations (NLI)")
ax2.legend()

st.pyplot(fig)
plt.clf()

st.subheader("1.4. Text Fluency and Quality")
col_div1, col_div2 = st.columns(2)
col_div1.metric("Unigram Diversity", f"""{data["unigrams_diversity"]:.2f}""")
col_div2.metric("Bigram Diversity", f"""{data["bigrams_diversity"]:.2f}""")

st.metric("Normalized Perplexity Score", f"""{data["normalized_perplexity"]:.2f}""")
st.info(f"""Perplexity (raw): {data["raw_perplexity"]:.2f}. Lower perplexity indicates higher fluency.""")

st.metric("Grammar Quality Score", f"""{data["grammar_score"]:.2f}""")
if data["grammar_errors"]:
    st.warning("Grammar Errors Found:")
    for error in data["grammar_errors"]:
        st.markdown(f"- **Issue**: {error['message']} | **Found**: `{error['context']['text']}`")
else:
    st.success("No grammar errors found by the checker.")


st.subheader("1.5. Instruction Adherence Check")
st.markdown("**Parsed Constraints:**")
st.json(data["parsed_constraints"])

eval_data = data["instruction_adherence"]

st.markdown("### LLM Evaluation Scores")
col_s1, col_s2, col_s3 = st.columns(3)
col_s4, col_s5, col_final = st.columns(3)

col_s1.metric("Quantity Score", f"{eval_data.get('quantity_score', 0):.2f}")
col_s2.metric("Format Score", f"{eval_data.get('format_score', 0):.2f}")
col_s3.metric("Length Score", f"{eval_data.get('length_score', 0):.2f}")
col_s4.metric("Style Score", f"{eval_data.get('style_score', 0):.2f}")
col_s5.metric("Do/Don't Score", f"{eval_data.get('do_dont_score', 0):.2f}")
col_final.metric("Final Score", f"{eval_data.get('final_score', 0):.2f}")

st.markdown("**Reasoning:**")
st.write(eval_data.get('reasoning', 'No reasoning provided.'))

st.subheader("1.6. Factual Accuracy Check")
st.metric("Factual Accuracy Score", f"""{data["factual_accuracy_score"]:.2f}""")

st.markdown("##### Extracted Claims:")
st.json(data['claims'])

st.markdown("##### Evidences:")
st.json(data['evidences'])

st.markdown("##### Factual Check Details:")
st.json(data["AI_as_judge_results"])

In [46]:
!pip install json-repair
import json_repair
import json

with open("data.json", "w") as f:
    json.dump(json_repair.loads(json.dumps(results)), f, indent=4)


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [47]:
!pip install pyngrok


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [53]:
from pyngrok import ngrok
import os

# Set the Streamlit port
port = 8501
ngrok.set_auth_token("32frCnc6yNene02xWYThb8jTgB6_2pNYAC6tYKi2W76FQ8Pf8")

# Start the tunnel
public_url = ngrok.connect(port)
print(public_url)

# Run the app
os.system(f"streamlit run app.py --server.port {port} &")

NgrokTunnel: "https://96d06aecc306.ngrok-free.app" -> "http://localhost:8501"


0

2025-09-14 12:34:04.432 Port 8501 is already in use



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
